In [1]:
#allows drawing digrams in jupyeter notebooks
%matplotlib inline
#loading tensorboard
%load_ext tensorboard
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#specific tensorflow libraries
import tensorflow as tf
from tensorflow.keras import optimizers, metrics
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [2]:
import datetime

In [3]:
from pandas.plotting import scatter_matrix

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [5]:
def Train_Test_XY_normalize_PCA(TrainX, TrainY, PCA_n = None):
    
    # normalize data
    scaler_new = StandardScaler()
    scaler_new.fit(TrainX)
    X_train_scale = scaler_new.transform(TrainX)
    print(f'X shape {X_train_scale.shape}')
    
    if PCA_n != None:
        # dimension reduction
        TrainX_PCA = Train_Test_XY_PCA(X_train_scale, PCA_n = PCA_n)
        print(f'X shape {TrainX_PCA.shape}')
    
        # normalize again
        scaler_new = StandardScaler()
        scaler_new.fit(TrainX_PCA)
        X_train_scale = scaler_new.transform(TrainX_PCA)
        
    
    X_train, X_test, y_train, y_test = train_test_split(X_train_scale,TrainY,
                                                       test_size=.2,
                                                       shuffle=True,
                                                       random_state=2020,
                                                       stratify=TrainY)
    
    
    return X_train, X_test, y_train, y_test

In [6]:
import tensorflow.keras.backend as kb
def custom_loss(y_actual,y_pred):
    custom_loss=kb.square(y_actual-y_pred)
    return custom_loss

In [7]:
model=Sequential()

model.add(layers.Dense(8, input_shape=(5,), activation='relu'))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(15, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

opt = optimizers.SGD(learning_rate=0.1)
# opt = optimizers.Adam(learning_rate=0.1)
# opt = optimizers.Nadam(learning_rate=0.001)

# model.compile(loss='mean_squared_error', optimizer=opt)
model.compile(loss='binary_crossentropy', optimizer=opt)
# model.compile(loss='huber', optimizer=opt)
# model.compile(loss='binary_focal_crossentropy', optimizer=opt)
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 48        
                                                                 
 dense_1 (Dense)             (None, 15)                135       
                                                                 
 dense_2 (Dense)             (None, 8)                 128       
                                                                 
 dense_3 (Dense)             (None, 15)                135       
                                                                 
 dense_4 (Dense)             (None, 8)                 128       
                                                                 
 dense_5 (Dense)             (None, 15)                135       
                                                                 
 dense_6 (Dense)             (None, 1)                 1

2022-06-01 20:48:26.101796: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


# simple NN model

In [8]:
## features selection from Team_EDA.ipynb
# see_column = np.array(['PTS', 'W','L', 'GF', 'GA', 'SRS', 
#                                   'SOS', 'PK%', 'S%', 'SV%', 'SAT%', 
#                                   'MsS', 'TA/GA'])
see_column = np.array(['W', 'L', 'GA', 'SAT%', 'TA/GA'])
column_home = np.array([i+'_H' for i in see_column])
column_away = np.array([i+'_A' for i in see_column])

column_name = np.concatenate((column_home,column_away))
column_name = np.concatenate((column_name,['class']))

#### Train NN model

In [9]:
n_year = 2
g = 0.9
g_list = np.array([0.9, 0.8, 0.7, 0.6, 0.5, 0.4, 0.3, 0.2, 0.1])
filename = 'trainXY/'+'nyear'+str(n_year)+'/'+str(20122021)+'_'+str(g)+'.csv'
# filename = 'trainXY/'+'nyear'+str(n_year)+'/'+str(20162021)+'_'+str(g)+'.csv'

Traindf = pd.read_csv(filename)[column_name]
Traindf = Traindf.dropna()
columns_name = Traindf.columns
Traindf_diff = pd.DataFrame(data = Traindf[column_home].to_numpy() 
                                    - Traindf[column_away].to_numpy(), 
                            columns = see_column)

TrainX_array = Traindf_diff.to_numpy()
TrainY_array = Traindf[columns_name[-1]].to_numpy()
print(sum(TrainY_array)/len(TrainY_array))

0.44669390787518576


In [10]:
X_train, X_test, y_train, y_test = Train_Test_XY_normalize_PCA(TrainX_array, 
                                                               TrainY_array)

X shape (10768, 5)


In [11]:
#creating unique name for tensorboard directory
log_dir = "logs/NN/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
#Tensforboard callback function
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

#concatentating input data
#x=np.transpose(x)
#training the netowork for 20 epochs with batch size of 5 (10/5*20 iterations)
model.fit(X_train,y_train,epochs=1000,batch_size=100,shuffle=True,callbacks=[tensorboard_callback])

Epoch 1/1000
87/87 [==============================] - 1s 1ms/step - loss: 0.6893
Epoch 2/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6831
Epoch 3/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6820
Epoch 4/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6813
Epoch 5/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6810
Epoch 6/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6806
Epoch 7/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6805
Epoch 8/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6800
Epoch 9/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6796
Epoch 10/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6794
Epoch 11/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6795
Epoch 12/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6793
Epoch 13/1000
87/87 [====

87/87 [==============================] - 0s 1ms/step - loss: 0.6736
Epoch 102/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6740
Epoch 103/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6741
Epoch 104/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6740
Epoch 105/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6736
Epoch 106/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6734
Epoch 107/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6735
Epoch 108/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6735
Epoch 109/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6731
Epoch 110/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6732
Epoch 111/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6732
Epoch 112/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6726
Epoch 113/1000
87/8

87/87 [==============================] - 0s 1ms/step - loss: 0.6676
Epoch 200/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6678
Epoch 201/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6682
Epoch 202/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6679
Epoch 203/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6675
Epoch 204/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6680
Epoch 205/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6670
Epoch 206/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6684
Epoch 207/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6667
Epoch 208/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6673
Epoch 209/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6674
Epoch 210/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6673
Epoch 211/1000
87/8

87/87 [==============================] - 0s 2ms/step - loss: 0.6631
Epoch 298/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6629
Epoch 299/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6631
Epoch 300/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6618
Epoch 301/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6629
Epoch 302/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6616
Epoch 303/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6639
Epoch 304/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6628
Epoch 305/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6618
Epoch 306/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6621
Epoch 307/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6624
Epoch 308/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6628
Epoch 309/1000
87/8

87/87 [==============================] - 0s 1ms/step - loss: 0.6584
Epoch 396/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6581
Epoch 397/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6588
Epoch 398/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6588
Epoch 399/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6573
Epoch 400/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6578
Epoch 401/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6595
Epoch 402/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6568
Epoch 403/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6589
Epoch 404/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6575
Epoch 405/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6568
Epoch 406/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6573
Epoch 407/1000
87/8

...

87/87 [==============================] - 0s 1ms/step - loss: 0.6509
Epoch 592/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6506
Epoch 593/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6515
Epoch 594/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6515
Epoch 595/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6501
Epoch 596/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6518
Epoch 597/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6518
Epoch 598/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6525
Epoch 599/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6512
Epoch 600/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6523
Epoch 601/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6491
Epoch 602/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6516
Epoch 603/1000
87/8

87/87 [==============================] - 0s 1ms/step - loss: 0.6490
Epoch 690/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6482
Epoch 691/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6489
Epoch 692/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6478
Epoch 693/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6500
Epoch 694/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6482
Epoch 695/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6486
Epoch 696/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6470
Epoch 697/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6481
Epoch 698/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6484
Epoch 699/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6475
Epoch 700/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6481
Epoch 701/1000
87/8

87/87 [==============================] - 0s 1ms/step - loss: 0.6477
Epoch 788/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6475
Epoch 789/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6458
Epoch 790/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6472
Epoch 791/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6451
Epoch 792/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6461
Epoch 793/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6442
Epoch 794/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6473
Epoch 795/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6452
Epoch 796/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6443
Epoch 797/1000
87/87 [==============================] - 0s 2ms/step - loss: 0.6447
Epoch 798/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6458
Epoch 799/1000
87/8

87/87 [==============================] - 0s 1ms/step - loss: 0.6441
Epoch 886/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6445
Epoch 887/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6458
Epoch 888/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6436
Epoch 889/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6423
Epoch 890/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6431
Epoch 891/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6427
Epoch 892/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6433
Epoch 893/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6455
Epoch 894/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6440
Epoch 895/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6441
Epoch 896/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6427
Epoch 897/1000
87/8

87/87 [==============================] - 0s 1ms/step - loss: 0.6413
Epoch 984/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6411
Epoch 985/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6429
Epoch 986/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6383
Epoch 987/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6444
Epoch 988/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6416
Epoch 989/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6399
Epoch 990/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6431
Epoch 991/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6423
Epoch 992/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6422
Epoch 993/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6424
Epoch 994/1000
87/87 [==============================] - 0s 1ms/step - loss: 0.6409
Epoch 995/1000
87/8

In [12]:
m = metrics.Accuracy()
mc = metrics.CategoricalAccuracy()

In [13]:
y_hat=model.predict(X_train)>0.5
y_hat=np.squeeze(y_hat)*1

270/270 [==============================] - 0s 567us/step


In [14]:
m.update_state(y_train, y_hat)
mc.update_state(y_train, y_hat)
print('home win rate = ', sum(y_hat)/len(y_hat))
print('Train data accuracy',m.result().numpy())
# print('Train data categorical accuracy',mc.result().numpy())
tf.math.confusion_matrix(y_train, y_hat)

home win rate =  0.4303459484560019
Train data accuracy 0.63071746


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[3246, 1520],
       [1661, 2187]], dtype=int32)>

In [15]:
y_hat=model.predict(X_test)>0.5
y_hat=np.squeeze(y_hat)*1

68/68 [==============================] - 0s 613us/step


In [16]:
m.update_state(y_test, y_hat)
print('home win rate = ',sum(y_hat)/len(y_hat))
print('Test data accuracy',m.result().numpy())
tf.math.confusion_matrix(y_test, y_hat)

home win rate =  0.4517177344475395
Test data accuracy 0.61088413


<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[682, 510],
       [499, 463]], dtype=int32)>